In [76]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
import time
import os
import cv2
import tensorflow as tf
import matplotlib.image as mpimg
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import fetch_olivetti_faces
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score
from skimage.transform import resize
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from skimage.color import rgb2gray
from tqdm.notebook import tqdm
from mpl_toolkits.mplot3d import Axes3D
from tensorflow.keras import layers, models
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Conv2D, MaxPooling2D, Flatten, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
gpus = tf.config.experimental.list_physical_devices('GPU') #with import tensorflow
tf.config.experimental.set_memory_growth(gpus[0], True) # me too

In [37]:
path = "./data/"
labels = []
file_id = []
full_names = []
lables = os.listdir(path)
for label in lables:
    data_names = os.listdir(path+ "/" +label)
    for name in data_names:
        labels.append(label)
        file_id.append(name)
        full_names.append(label + "/" + name)

full_names

['3/110.pickle',
 '3/116.pickle',
 '3/145.pickle',
 '3/0.pickle',
 '3/99.pickle',
 '3/125.pickle',
 '3/211.pickle',
 '3/16.pickle',
 '3/158.pickle',
 '3/140.pickle',
 '3/1.pickle',
 '3/170.pickle',
 '3/243.pickle',
 '3/86.pickle',
 '3/178.pickle',
 '3/109.pickle',
 '3/101.pickle',
 '3/177.pickle',
 '3/22.pickle',
 '3/184.pickle',
 '3/13.pickle',
 '3/228.pickle',
 '3/73.pickle',
 '3/179.pickle',
 '3/168.pickle',
 '3/28.pickle',
 '3/217.pickle',
 '3/21.pickle',
 '3/218.pickle',
 '3/197.pickle',
 '3/152.pickle',
 '3/156.pickle',
 '3/11.pickle',
 '3/96.pickle',
 '3/194.pickle',
 '3/188.pickle',
 '3/189.pickle',
 '3/95.pickle',
 '3/210.pickle',
 '3/169.pickle',
 '3/61.pickle',
 '3/134.pickle',
 '3/151.pickle',
 '3/157.pickle',
 '3/191.pickle',
 '3/203.pickle',
 '3/6.pickle',
 '3/35.pickle',
 '3/104.pickle',
 '3/90.pickle',
 '3/153.pickle',
 '3/8.pickle',
 '3/172.pickle',
 '3/160.pickle',
 '3/241.pickle',
 '3/139.pickle',
 '3/71.pickle',
 '3/171.pickle',
 '3/200.pickle',
 '3/173.pickle',
 '3

In [38]:
xyz_df = pd.read_csv("./skeleton_data.csv")
xyz_df.head(40)


,x,y,z,label
0,0.000000,0.000000,0.000000,3
1,0.054078,0.027680,0.018147,3
2,0.116968,0.079316,0.006362,3
3,0.163603,0.106157,-0.009863,3
4,0.199006,0.094051,-0.025528,3
5,0.128502,0.101975,-0.038733,3
6,0.144138,0.202252,-0.056050,3
7,0.153579,0.260013,-0.062242,3
8,0.162536,0.309633,-0.067903,3
9,0.101091,0.102251,-0.062031,3


In [39]:
labels = xyz_df["label"]
xyz_df = xyz_df.drop("label", axis=1)

In [69]:
xyz_df.shape

(39732, 3)

In [73]:
arr = np.array([xyz_df.iloc[i:i+21, :].to_numpy() for i in range(0, len(xyz_df), 21)])
arr.shape

(1892, 21, 3)

In [100]:
new_labels = []
for i in range(0, len(labels), 21):
    new_labels.append(labels[i])
    print(i)
print(len(new_labels))

0
21
42
63
84
105
126
147
168
189
210
231
252
273
294
315
336
357
378
399
420
441
462
483
504
525
546
567
588
609
630
651
672
693
714
735
756
777
798
819
840
861
882
903
924
945
966
987
1008
1029
1050
1071
1092
1113
1134
1155
1176
1197
1218
1239
1260
1281
1302
1323
1344
1365
1386
1407
1428
1449
1470
1491
1512
1533
1554
1575
1596
1617
1638
1659
1680
1701
1722
1743
1764
1785
1806
1827
1848
1869
1890
1911
1932
1953
1974
1995
2016
2037
2058
2079
2100
2121
2142
2163
2184
2205
2226
2247
2268
2289
2310
2331
2352
2373
2394
2415
2436
2457
2478
2499
2520
2541
2562
2583
2604
2625
2646
2667
2688
2709
2730
2751
2772
2793
2814
2835
2856
2877
2898
2919
2940
2961
2982
3003
3024
3045
3066
3087
3108
3129
3150
3171
3192
3213
3234
3255
3276
3297
3318
3339
3360
3381
3402
3423
3444
3465
3486
3507
3528
3549
3570
3591
3612
3633
3654
3675
3696
3717
3738
3759
3780
3801
3822
3843
3864
3885
3906
3927
3948
3969
3990
4011
4032
4053
4074
4095
4116
4137
4158
4179
4200
4221
4242
4263
4284
4305
4326
4347
4368
4389
4410

In [99]:
new_labels

[3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,


In [101]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(new_labels)
labels_encoded = encoder.transform(new_labels)
labels_encoded[:3], encoder.classes_

(array([3, 3, 3]), array([0, 1, 2, 3, 4, 5, 6, 7, 8]))

In [102]:
############# 라벨이 각 x,y,z 값마다 달려있어서 개수가 안맞음 수정필요
X_train, X_test, y_train, y_test = train_test_split(arr, labels_encoded, test_size=0.2,
                                                    random_state=13, stratify=labels_encoded)
X_train.shape, X_test.shape

((1513, 21, 3), (379, 21, 3))

In [103]:
# 모델 생성
model = Sequential()

# Conv2D 레이어 추가 (입력 데이터 형태를 (21, 3, 1)로 지정)
model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(21, 3, 1)))
model.add(Activation("relu"))


model.add(Conv2D(32, (3, 3)))
model.add(Activation("relu"))


model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))


model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation("sigmoid"))

# 모델 요약 출력
model.summary()

ValueError: One of the dimensions in the output is <= 0 due to downsampling in conv2d_18. Consider increasing the input size. Received input shape [None, 19, 1, 32] which would produce output shape with a zero or negative value in a dimension.